# BUILD A CHATBOT USING LANGCHAIN

https://python.langchain.com/docs/tutorials/chatbot/

## LangChain and LangGraph installation

```
pip install langchain-core langgraph>0.2.27
```

## LangSmith for tracing

```
LANGCHAIN_TRACING_V2=true  
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  
LANGCHAIN_API_KEY="xxx"  
LANGCHAIN_PROJECT="xxx"
```

## LLM installation

```
pip install -qU langchain-openai                  #openai
pip install -qU langchain-anthropic               #anthropic
pip install -qU langchain-google-vertexai         #google
pip install -qU langchain-groq                    #groq
```

# QUICKSTART

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

model = ChatOpenAI(model="gpt-4o-mini")

model.invoke(
    [
        HumanMessage(content="Hi! My name is Dd."),
        AIMessage(content="Hello Dd! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Dd! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 38, 'total_tokens': 51, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-d85d4387-7dff-4760-ab2a-42ffed47efa2-0', usage_metadata={'input_tokens': 38, 'output_tokens': 13, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

# MESSAGE PERSISTENCE IN LANGGRAPH

In [2]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Config into runnable
config = {"configurable": {"thread_id": "dd001"}}

# Invoke the application

query = "Hi! My name is Dd."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Dd! How can I assist you today?


In [3]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Dd! How can I help you today?


# PROMPT TEMPLATE - SIMPLE

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [5]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    chain = prompt | model                                               # The new chain
    response = chain.invoke(state)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Config into runnable
config = {"configurable": {"thread_id": "dd002"}}

# Invoke the application

query = "Hi! My name is Dd."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Ahoy, Dd! Welcome aboard me ship! What be it ye be seekin’ today? Arrr!


# PROMPT TEMPLATE - PARAMETERS

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [7]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):                                                       # The new typeddict
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "dd003"}}
query = "Hi! My name is Dd."
language = "Tagalog"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},                    # Includes the parameters
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Kamusta, Dd! Ano ang maitutulong ko sa iyo ngayon?


# MANAGING CONVERSATION HISTORY

- trimming - https://python.langchain.com/docs/how_to/trim_messages/
- filtering - https://python.langchain.com/docs/how_to/filter_messages/
- merging - https://python.langchain.com/docs/how_to/merge_message_runs/

## Trimming

In [8]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! My name is Dd."),
    AIMessage(content="hi!"),
    HumanMessage(content="I like rocky road ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 x 3"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 x 3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [9]:
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])                          # Trim the messages
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "dd004"}}
query = "What math problem did I ask?"
language = "Tagalog"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tinanong mo kung ano ang 2 x 3.


In [11]:
config = {"configurable": {"thread_id": "dd004"}}
query = "Ano ang sagot?"
language = "Tagalog"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ang sagot sa 2 x 3 ay 6.


# Filtering

In [12]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    filter_messages,
)

messages = [
    SystemMessage("you are a good assistant", id="1"),
    HumanMessage("example input", id="2", name="example_user"),
    AIMessage("example output", id="3", name="example_assistant"),
    HumanMessage("real input", id="4", name="dd"),
    AIMessage("real output", id="5", name="albert"),
]

In [13]:
filter_messages(messages, include_types="human")

[HumanMessage(content='example input', additional_kwargs={}, response_metadata={}, name='example_user', id='2'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='dd', id='4')]

In [14]:
filter_messages(messages, exclude_names=["example_user", "example_assistant"])

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}, id='1'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='dd', id='4'),
 AIMessage(content='real output', additional_kwargs={}, response_metadata={}, name='albert', id='5')]

In [15]:
filter_messages(messages, include_types=[HumanMessage, AIMessage], exclude_ids=["3"])

[HumanMessage(content='example input', additional_kwargs={}, response_metadata={}, name='example_user', id='2'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='dd', id='4'),
 AIMessage(content='real output', additional_kwargs={}, response_metadata={}, name='albert', id='5')]

## Merging

In [16]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    merge_message_runs,
)

messages = [
    SystemMessage("you're a good assistant."),
    SystemMessage("you always respond with a joke."),
    HumanMessage([{"type": "text", "text": "i wonder why it's called langchain"}]),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    AIMessage("Why, he's probably chasing after the last cup of coffee in the office!"),
]

merged = merge_message_runs(messages)                                        # Merge messages
print("\n\n".join([repr(x) for x in merged]))

SystemMessage(content="you're a good assistant.\nyou always respond with a joke.", additional_kwargs={}, response_metadata={})

HumanMessage(content=[{'type': 'text', 'text': "i wonder why it's called langchain"}, 'and who is harrison chasing anyways'], additional_kwargs={}, response_metadata={})

AIMessage(content='Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!\nWhy, he\'s probably chasing after the last cup of coffee in the office!', additional_kwargs={}, response_metadata={})


# STREAMING

In [17]:
config = {"configurable": {"thread_id": "dd005"}}
query = "Hi My name is Dd, please tell me a joke."
language = "Tagalog"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|K|um|usta|,| D|d|!| Nar|ito| ang| isang| biro| para| sa| iyo|:

|B|akit| hindi| nag|tag|ump|ay| ang| skeleton| sa| pags|ali| sa| mga| party|?

|D|ahil| wala| siyang| "|g|uts|"!| 😄||